# Batch evaluation with your custom evaluators
The following sample shows the basic way to evaluate a Generative AI application in your development environment with your custom evaluators.

> ✨ ***Note*** <br>
> Please check the reference document before you get started - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk

## 🔨 Current Support and Limitations (as of 2025-01-14) 
- Check the region support for the Azure AI Evaluation SDK. https://learn.microsoft.com/en-us/azure/ai-studio/concepts/evaluation-metrics-built-in?tabs=warning#region-support

### Region support for evaluations
| Region              | Hate and Unfairness, Sexual, Violent, Self-Harm, XPIA, ECI (Text) | Groundedness (Text) | Protected Material (Text) | Hate and Unfairness, Sexual, Violent, Self-Harm, Protected Material (Image) |
|---------------------|------------------------------------------------------------------|---------------------|----------------------------|----------------------------------------------------------------------------|
| North Central US    | no                                                               | no                  | no                         | yes                                                                        |
| East US 2           | yes                                                              | yes                 | yes                        | yes                                                                        |
| Sweden Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| US North Central    | yes                                                              | no                  | yes                        | yes                                                                        |
| France Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| Switzerland West    | yes                                                              | no                  | no                         | yes                                                                        |

### Region support for adversarial simulation
| Region            | Adversarial Simulation (Text) | Adversarial Simulation (Image) |
|-------------------|-------------------------------|---------------------------------|
| UK South          | yes                           | no                              |
| East US 2         | yes                           | yes                             |
| Sweden Central    | yes                           | yes                             |
| US North Central  | yes                           | yes                             |
| France Central    | yes                           | no                              |


## ✔️ Pricing and billing
- Effective 1/14/2025, Azure AI Safety Evaluations will no longer be free in public preview. It will be billed based on consumption as following:

| Service Name              | Safety Evaluations       | Price Per 1K Tokens (USD) |
|---------------------------|--------------------------|---------------------------|
| Azure Machine Learning    | Input pricing for 3P     | $0.02                     |
| Azure Machine Learning    | Output pricing for 3P    | $0.06                     |
| Azure Machine Learning    | Input pricing for 1P     | $0.012                    |
| Azure Machine Learning    | Output pricing for 1P    | $0.012                    |


In [ ]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration,
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
    F1ScoreEvaluator,
    RetrievalEvaluator,
)

from azure.ai.ml import MLClient


load_dotenv(override=True)

## 🚀 Run Evaluators in Azure Cloud (azure.ai.evaluation.evaluate)

In [ ]:
credential = DefaultAzureCredential()

azure_ai_project_conn_str = os.environ.get("AZURE_AI_PROJECT_CONN_STR")
subscription_id = azure_ai_project_conn_str.split(";")[1]
resource_group_name = azure_ai_project_conn_str.split(";")[2]
project_name = azure_ai_project_conn_str.split(";")[3]

azure_ai_project_dict = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}

azure_ai_project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=azure_ai_project_conn_str
)


model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai",
}

ml_client = MLClient(credential, subscription_id, resource_group_name, project_name)

## set the evaluation dataset
- Use your query data set on your storage account. These response records serve as the seed for creating assessments. 

In [ ]:
from openai import AzureOpenAI


aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

try:
    client = AzureOpenAI(
        azure_endpoint=aoai_api_endpoint,
        api_key=aoai_api_key,
        api_version=aoai_api_version,
    )

    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")

except (ValueError, TypeError) as e:
    print(e)

### You have all the data (Query + Context + Response + Ground Truth)
- Assume that you already have all the data (Query + Context + Response + Ground Truth) in a local folder

In [ ]:
input_path = "./data/sythetic_evaluation_data.jsonl"

## 🚀 Run Evaluators for local and upload to cloud (azure.ai.evaluation.evaluate)
- set up your custom GroundnessEvaluator, ExactMatchEvaluator for Local environment with the following steps.

In [ ]:
import os
from typing_extensions import override

class CustomGroundednessEvaluator(GroundednessEvaluator):
    """
    Evaluates groundedness score for a given query (optional), response, and context or a multi-turn conversation,
    including reasoning.

    The groundedness measure assesses the correspondence between claims in an AI-generated answer and the source
    context, making sure that these claims are substantiated by the context. Even if the responses from LLM are
    factually correct, they'll be considered ungrounded if they can't be verified against the provided sources
    (such as your input source or your database). Use the groundedness metric when you need to verify that
    AI-generated responses align with and are validated by the provided context.

    Groundedness scores range from 0.0 to 1.0, with 0.0 being the least grounded and 1.0 being the grounded.

    :param model_config: Configuration for the Azure OpenAI model.
    :type model_config: Union[~azure.ai.evaluation.AzureOpenAIModelConfiguration,
        ~azure.ai.evaluation.OpenAIModelConfiguration]

    .. admonition:: Example:

        .. literalinclude:: ../samples/evaluation_samples_evaluate.py
            :start-after: [START groundedness_evaluator]
            :end-before: [END groundedness_evaluator]
            :language: python
            :dedent: 8
            :caption: Initialize and call a GroundednessEvaluator.

    .. note::

        To align with our support of a diverse set of models, an output key without the `gpt_` prefix has been added.
        To maintain backwards compatibility, the old key with the `gpt_` prefix is still be present in the output;
        however, it is recommended to use the new key moving forward as the old key will be deprecated in the future.
    """
    current_dir = os.getcwd()
    
    # need to set the new prompty file path because the variables are still used in the parent call method
    _PROMPTY_FILE_NO_QUERY = os.path.join(current_dir, 'custom-groundedness.prompty')
    _PROMPTY_FILE_WITH_QUERY = os.path.join(current_dir, 'custom-groundedness.prompty')

    
    @override
    def __init__(self, model_config):
        
        super().__init__(model_config)
        current_dir = os.getcwd()
        prompty_path = os.path.join(current_dir, "custom-groundedness.prompty")  # Default to no query
        super(GroundednessEvaluator, self).__init__(model_config=model_config, prompty_file=prompty_path, result_key="custom-groundedness")

In [ ]:
from ExactMatchEvaluator._exact_match import ExactMatchEvaluator

# check the source code of the ExactMatchEvaluator
# This is a custom evaluator that relates to the task of evaluating in local environment
'''
    class ExactMatchEvaluator(EvaluatorBase):
    """
    Evaluates whether the response exactly matches the ground truth.

    This evaluator returns a score of 1.0 if the response is identical to the ground truth,
    and 0.0 otherwise. It is useful for tasks that require strict correctness such as factual QA.

    Example:
        evaluator = ExactMatchEvaluator()
        result = evaluator(ground_truth="Hello, world!", response="Hello, world!")
        print(result)  # {'exact_match_score': 1.0}
    """

    id = "update with your azure ml asset id"
    """Evaluator identifier, experimental and to be used only with evaluation in cloud."""

    @override
    def __init__(self):
        super().__init__()

    @override
    async def _do_eval(self, eval_input: Dict) -> Dict[str, float]:
        """Evaluate whether the response matches the ground truth exactly."""
        ground_truth = eval_input["ground_truth"].strip()
        response = eval_input["response"].strip()
        
        score = 1.0 if ground_truth == response else 0.0

        return {
            "exact_match_score": score,
        }

    @overload
    def __call__(self, *, ground_truth: str, response: str):
        """
        Evaluate whether the response matches the ground truth exactly.

        :keyword response: The response to be evaluated.
        :paramtype response: str
        :keyword ground_truth: The ground truth to be compared against.
        :paramtype ground_truth: str
        :return: The exact match score.
        :rtype: Dict[str, float]
        """

    @override
    def __call__(self, *args, **kwargs):
        """Evaluate whether the response matches the ground truth exactly."""
        return super().__call__(*args, **kwargs)

'''

In [ ]:
exactMatchEvaluator = ExactMatchEvaluator()

exactMatch = exactMatchEvaluator(
    ground_truth="What is the speed of light?", response="What is the speed of light?"
)

exactMatch

In [ ]:
input_path = input_path  
output_path = "./data/local_upload_cloud_evaluation_output.json"


# https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/flow-evaluate-sdk

retrieval_evaluator = RetrievalEvaluator(model_config)
custom_groundedness_evaluator = CustomGroundednessEvaluator(model_config)
relevance_evaluator = RelevanceEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)
exactMatchEvaluator = ExactMatchEvaluator()

column_mapping = {
    "query": "${data.query}",
    "ground_truth": "${data.ground_truth}",
    "response": "${data.response}",
    "context": "${data.context}",
}

In [ ]:
import datetime

result = evaluate(
    evaluation_name=f"custom_evaluation_local_upload_cloud_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    data=input_path,
    evaluators={
        "retrieval": retrieval_evaluator,
        "custom-groundedness": custom_groundedness_evaluator,
        "relevance": relevance_evaluator,
        "similarity": similarity_evaluator,
        "exact_match": exactMatchEvaluator,
    },
    evaluator_config={
        "retrieval": {"column_mapping": column_mapping},
        "custom-groundedness": {"column_mapping": column_mapping},
        "relevance": {"column_mapping": column_mapping},
        "similarity": {"column_mapping": column_mapping},
        "exact_match": {"column_mapping": column_mapping},
    },
    azure_ai_project=azure_ai_project_dict,
    output_path=output_path,
)

### Visualize evaluation Results as html
- You can visualize the evaluation results as HTML with the following steps.

In [ ]:
import json
import matplotlib.pyplot as plt
import base64
from io import BytesIO
import numpy as np
from datetime import datetime
import json

def generate_evaluation_report(data_file):
    import matplotlib.pyplot as plt

    # Load JSON file
    with open(data_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    rows = data.get('rows', [])

    # Define score ranges
    score_range = [0, 1, 2, 3, 4, 5]
    freq_retrieval = {score: 0 for score in score_range}
    freq_relevance = {score: 0 for score in score_range}
    freq_similarity = {score: 0 for score in score_range}
    freq_groundedness = {score: 0 for score in score_range}
    freq_exact_match = {score: 0 for score in [0, 1]}

    # Count occurrences of each score
    for row in rows:
        retrieval_score = row.get('outputs.retrieval.retrieval', 0)
        relevance_score = row.get('outputs.relevance.relevance', 0)
        similarity_score = row.get('outputs.similarity.similarity', 0)
        groundedness_score = row.get('outputs.custom-groundedness.custom-groundedness', 0)
        exact_match_score = row.get('outputs.exact_match.exact_match_score', 0)
        
        if retrieval_score in freq_retrieval:
            freq_retrieval[retrieval_score] += 1
        if relevance_score in freq_relevance:
            freq_relevance[relevance_score] += 1
        if similarity_score in freq_similarity:
            freq_similarity[similarity_score] += 1
        if groundedness_score in freq_groundedness:
            freq_groundedness[groundedness_score] += 1
        if exact_match_score in freq_exact_match:
            freq_exact_match[exact_match_score] += 1

    # Function to generate bar chart
    def generate_chart(freq_dict, title):
        fig, ax = plt.subplots()
        x = np.arange(len(freq_dict))
        ax.bar(x, [freq_dict.get(score, 0) for score in freq_dict], width=0.5, label=title)
        ax.set_xticks(x)
        ax.set_xticklabels(list(freq_dict.keys()))
        ax.set_xlabel('Score')
        ax.set_ylabel('Frequency')
        ax.set_title(title)
        ax.legend()
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        chart_data = base64.b64encode(buf.read()).decode('utf-8')
        plt.close(fig)
        return chart_data

    # Generate charts
    retrieval_chart = generate_chart(freq_retrieval, 'Retrieval Score Distribution')
    exact_match_chart = generate_chart(freq_exact_match, 'Exact Match Score Distribution')

    # Generate combined response chart
    def generate_response_chart():
        fig, ax = plt.subplots()
        x = np.arange(len(score_range))
        width = 0.3
        ax.bar(x - width, [freq_relevance[score] for score in score_range], width, label='Relevance')
        ax.bar(x, [freq_similarity[score] for score in score_range], width, label='Similarity')
        ax.bar(x + width, [freq_groundedness[score] for score in score_range], width, label='Custom-Groundedness')
        ax.set_xticks(x)
        ax.set_xticklabels(score_range)
        ax.set_xlabel('Score')
        ax.set_ylabel('Frequency')
        ax.set_title('Response Score Distribution')
        ax.legend()
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        chart_data = base64.b64encode(buf.read()).decode('utf-8')
        plt.close(fig)
        return chart_data

    response_chart = generate_response_chart()

    # Generate HTML table
    table_html = '<table border="1" style="border-collapse: collapse;"><tr><th>Query</th><th>Response</th><th>Relevance</th><th>Similarity</th><th>Custom-Groundedness</th><th>Exact Match</th></tr>'
    for row in rows:
        table_html += f"<tr><td>{row.get('inputs.query', '')}</td><td>{row.get('inputs.response', '')}</td><td>{row.get('outputs.relevance.relevance', '')}</td><td>{row.get('outputs.similarity.similarity', '')}</td><td>{row.get('outputs.custom-groundedness.custom-groundedness', '')}</td><td>{row.get('outputs.exact_match.exact_match_score', '')}</td></tr>"
    table_html += '</table>'

    # Generate HTML content
    html_content = f"""
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Evaluation Results</title>
        <style>
            .image-container {{ display: flex; flex-wrap: wrap; justify-content: space-around; }}
            .image-container div {{ margin: 10px; text-align: center; }}
            img {{ max-width: 100%; height: auto; }}
        </style>
    </head>
    <body>
        <h1>Evaluation Results</h1>
        <div class="image-container">
            <div><h2>Retrieval Score Distribution</h2><img src="data:image/png;base64,{retrieval_chart}"/></div>
            <div><h2>Response Score Distribution</h2><img src="data:image/png;base64,{response_chart}"/></div>
            <div><h2>Exact Match Score Distribution</h2><img src="data:image/png;base64,{exact_match_chart}"/></div>
        </div>
        <h2>Results Table</h2>
        {table_html}
    </body>
    </html>
    """

    # Save HTML file with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'{timestamp}_evaluation_results.html'
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"HTML file '{filename}' generated.")



In [ ]:
generate_evaluation_report('data/local_upload_cloud_evaluation_output.json')

!["evaluation_result"](images/evaluation_result_local_upload_cloud1.png)

## 🚀 Run Evaluators in Azure Cloud (azure.ai.projects.models.Evaluation)
- set up your custom evaluator for cloud environment with the following steps.

In [ ]:
import logging
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError
from azure.ai.ml.entities import Model

logger = logging.getLogger(__name__)


def get_or_create_model_asset(
    ml_client,
    model_name,
    model_dir="ExactMatchEvaluator",
    model_type="custom_model",
    update=True,
):

    try:
        latest_model_version = max(
            [int(m.version) for m in ml_client.models.list(name=model_name)]
        )
        if update:
            raise ResourceExistsError("Found Model asset, but will update the Model.")
        else:
            model_asset = ml_client.models.get(
                name=model_name, version=latest_model_version
            )
            logger.info(f"Found Model asset: {model_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:

        logger.info(f"Exception: {e}")
        run_model = Model(
            name=model_name,
            path=model_dir,
            description="Model created from run.",
            properties={
                "_default-display-file": "./ExactMatchEvaluator/_exact_match_cloud.py",
                "is-evaluator": True,
                "is-promptflow": True,
                "show-artifact": True,
            },
            type=model_type,
        )
        model_asset = ml_client.models.create_or_update(run_model)
        logger.info(f"Created Model asset: {model_name}")

    return model_asset

In [ ]:
exact_match_model_asset = get_or_create_model_asset(
    ml_client,
    model_name="ExactMatchEvaluator",
    model_dir="ExactMatchEvaluator",
    model_type="custom_model",
    update=True,
)

### Get the azure ml asset id from Azure Machine Learning
- You can get the asset id from Azure Machine Learning workspace.
![azureml asset id](images/copy_azure_ml_asset_id.png)

In [ ]:
# id for each evaluator can be found in your AI Studio registry - please see documentation for more information
# init_params is the configuration for the model to use to perform the evaluation
# data_mapping is used to map the output columns of your query to the names required by the evaluator
# Evaluator parameter format - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#evaluator-parameter-format
evaluators_cloud = {
    "exact_match": EvaluatorConfiguration(
        # need to check the azure machine learning service to find the appropriate id
        # id="azureml://locations/swedencentral/workspaces/c68cb823-8b8f-4f88-bcc0-2c9f49675905/models/ExactMatchEvaluator/versions/5",
        id="your azureml asset id for ExactMatchEvaluator",
        data_mapping={
            "query": "${data.query}",
            "response": "${data.predicted}",
            "ground_truth": "${data.actual}",
        },
    ),
    "similarity": EvaluatorConfiguration(
        # currently bug in the SDK, please use the id below
        # id=SimilarityEvaluator.id,
        id="azureml://registries/azureml/models/Similarity-Evaluator/versions/3",
        init_params={"model_config": model_config},
        data_mapping={
            "query": "${data.query}",
            "response": "${data.predicted}",
            "ground_truth": "${data.actual}",
        },
    ),
}

### Data
- The following code demonstrates how to upload the data for evaluation to your Azure AI project. Below we use evaluate_test_data.jsonl which exemplifies LLM-generated data in the query-response format expected by the Azure AI Evaluation SDK. For your use case, you should upload data in the same format, which can be generated using the Simulator from Azure AI Evaluation SDK.

- Alternatively, if you already have an existing dataset for evaluation, you can use that by finding the link to your dataset in your registry or find the dataset ID.

In [ ]:
# # Upload data for evaluation
data_id, _ = azure_ai_project_client.upload_file("data/custom_data.jsonl")
# data_id = "azureml://registries/<registry>/data/<dataset>/versions/<version>"
# To use an existing dataset, replace the above line with the following line
# data_id = "<dataset_id>"

### Configure Evaluators to Run
- The code below demonstrates how to configure the evaluators you want to run. In this example, we use the F1ScoreEvaluator, RelevanceEvaluator and the ViolenceEvaluator, but all evaluators supported by Azure AI Evaluation are supported by cloud evaluation and can be configured here. You can either import the classes from the SDK and reference them with the .id property, or you can find the fully formed id of the evaluator in the AI Studio registry of evaluators, and use it here. 

In [ ]:
evaluation = Evaluation(
    display_name=f"custom_evaluation_cloud_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    description="Cloud Evaluation of dataset",
    data=Dataset(id=data_id),
    evaluators=evaluators_cloud,
)

# Create evaluation
evaluation_response = azure_ai_project_client.evaluations.create(
    evaluation=evaluation,
)

In [ ]:
from tqdm import notebook
import time


# Monitor the status of the run_result
def monitor_status(project_client: AIProjectClient, evaluation_response_id: str):
    with notebook.tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = project_client.evaluations.get(evaluation_response_id).status
        if status == "Queued":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            notebook.tqdm.write(f"Current Status: {status}")
            time.sleep(10)
            status = project_client.evaluations.get(evaluation_response_id).status
        while pbar.n < 3:
            pbar.update(1)
        notebook.tqdm.write("Operation Completed")

In [ ]:
monitor_status(azure_ai_project_client, evaluation_response.id)

### Add custom chart 
- After running the evaluation, you can add custom chart to visualize the evaluation results in Azure AI Foundry. 
![custom chart](images/exact_match_chart.png)